In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Flatten, GlobalAveragePooling2D, Dense, Normalization, BatchNormalization, Dropout, LeakyReLU, ELU, Lambda
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import EfficientNetB3, efficientnet_v2
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import RMSprop, Adam, SGD, Adagrad, Adadelta
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.metrics import AUC

dataset_folder = "/content/drive/MyDrive/Agricultural-crops"

image_paths = []
labels = []
classes = os.listdir(dataset_folder)
n_classes = len(classes)
for c in classes:
    class_folder = dataset_folder + "/" + c
    for image in os.listdir(class_folder):
        image_path = class_folder + "/" + image
        image_paths.append(image_path)
        labels.append(c)

df = pd.DataFrame({"image_path": image_paths, "label": labels})

train_df, val_df = train_test_split(df, test_size=0.30, stratify=df["label"], random_state=666)

datagen =ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=(0.8, 1.2),
    shear_range=0.1,
    channel_shift_range=0.1
    )

ts = 150 ;

train_generator = datagen.flow_from_dataframe(
    train_df,
    x_col="image_path",
    y_col="label",
    target_size=(ts, ts),
    class_mode="categorical",
    shuffle=False
)

val_generator = datagen.flow_from_dataframe(
    val_df,
    x_col="image_path",
    y_col="label",
    target_size=(ts, ts),
    class_mode="categorical",
    shuffle=False
)
tf.test.is_gpu_available()

Found 580 validated image filenames belonging to 30 classes.
Found 249 validated image filenames belonging to 30 classes.


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [ ]:
df

,image_path,label
0,/content/drive/MyDrive/Agricultural-crops/tea/...,tea
1,/content/drive/MyDrive/Agricultural-crops/tea/...,tea
2,/content/drive/MyDrive/Agricultural-crops/tea/...,tea
3,/content/drive/MyDrive/Agricultural-crops/tea/...,tea
4,/content/drive/MyDrive/Agricultural-crops/tea/...,tea
...,...,...
824,/content/drive/MyDrive/Agricultural-crops/cott...,cotton
825,/content/drive/MyDrive/Agricultural-crops/cott...,cotton
826,/content/drive/MyDrive/Agricultural-crops/cott...,cotton
827,/content/drive/MyDrive/Agricultural-crops/cott...,cotton


In [ ]:
import time
from keras.applications.vgg19 import VGG19

pre_trained_model = VGG19(include_top=False, weights='imagenet', input_shape=(150, 150, 3))
for layer in pre_trained_model.layers:
    layer.trainable = False

last_output = pre_trained_model.output

from tensorflow.keras import Model

def transfer_learning(last_output, pre_trained_model):
    x = tf.keras.layers.Flatten()(last_output)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(30, activation='softmax')(x)
    model = Model(inputs=pre_trained_model.input, outputs=x)
    return model

model_2 = transfer_learning(last_output, pre_trained_model)

model_2.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.8):
            print("\nReached 80% accuracy so cancelling training!")
            self.model.stop_training = True
callbacks = myCallback()
inicio = time.time()
history_2 = model_2.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=callbacks
)
# Grava o tempo final
fim = time.time()

# Calcula o tempo total decorrido
tempo_total = fim - inicio
print(f"Tempo de execução: {tempo_total} segundos")


Epoch 1/20
19/19 [==============================] - 282s 15s/step - loss: 38.3594 - accuracy: 0.1034 - val_loss: 16.9958 - val_accuracy: 0.2169
Epoch 2/20
19/19 [==============================] - 274s 15s/step - loss: 11.5525 - accuracy: 0.3603 - val_loss: 8.4836 - val_accuracy: 0.2651
Epoch 3/20
19/19 [==============================] - 275s 15s/step - loss: 4.7603 - accuracy: 0.4655 - val_loss: 4.5926 - val_accuracy: 0.3414
Epoch 4/20
19/19 [==============================] - 328s 18s/step - loss: 2.6662 - accuracy: 0.4983 - val_loss: 3.9498 - val_accuracy: 0.2691
Epoch 5/20
19/19 [==============================] - 276s 15s/step - loss: 2.2513 - accuracy: 0.5000 - val_loss: 3.1357 - val_accuracy: 0.3454
Epoch 6/20
19/19 [==============================] - 331s 18s/step - loss: 1.8120 - accuracy: 0.5690 - val_loss: 3.2083 - val_accuracy: 0.3173
Epoch 7/20
19/19 [==============================] - 274s 15s/step - loss: 1.8121 - accuracy: 0.5672 - val_loss: 3.3690 - val_accuracy: 0.3012
Epo

In [ ]:
import matplotlib.pyplot as plt
print('Train AUC: {:.2f}'.format(history_2.history['loss'][-1]))
print('Validation AUC: {:.2f}'.format(history_2.history['val_loss'][-1]))

plt.plot(history_2.history['val_loss'])
plt.plot(history_2.history['loss'])
plt.title('Perda x Épocas em Rede Neural pre-treinada VGG19')
plt.xlabel('Épocas')
plt.ylabel('Perda')
plt.legend(['Treino', 'Validação'], loc='upper right')
plt.savefig('plot_salvato.png')
plt.show()

In [ ]:

plt.plot(history_2.history['accuracy'])
plt.plot(history_2.history['val_accuracy'])
plt.title('Acurácia x Épocas em Rede Neural pre-treinada VGG19')
plt.xlabel('Épocas')
plt.ylabel('Acurácia')
plt.legend(['Treino', 'Validação'], loc='upper right')
plt.show()